In [2]:
# imports modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle
import os
%matplotlib inline

In [4]:
dataset=pd.read_csv('../Data/Churn_Modelling.csv')
# drop irrelevent data
dataset = dataset.drop(columns=['RowNumber','CustomerId','Surname'], axis=1)

gender_encoder = LabelEncoder()
geo_oh_encoder = OneHotEncoder(sparse_output=False)
dataset['Gender'] = gender_encoder.fit_transform(dataset['Gender'])
encoded_data = geo_oh_encoder.fit_transform(dataset['Geography'].values.reshape(-1, 1))
geo_oh_encoder.get_feature_names_out(['Geography'])
geography_encoded_df = pd.DataFrame(encoded_data, columns=geo_oh_encoder.get_feature_names_out(['Geography']))

In [5]:
dataset = pd.concat([dataset.drop(['Geography'], axis=1), geography_encoded_df], axis=1)
dataset

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [8]:
# Save the gender encoder and scale
with open('../pickle_files/gender_label_encoder_regression.pkl', 'wb') as file:
    pickle.dump(gender_encoder, file)
    
# Save the geography encoder and scale
with open('../pickle_files/geography_oh_label_encoder_regression.pkl', 'wb') as file:
    pickle.dump(geo_oh_encoder, file)

In [7]:
### Dependent and Independent features selection
X = dataset.drop(columns=['Exited'], axis=1)
y = dataset['Exited']

### Split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

# Scales the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
# save this scaler file as pickle
with open('../pickle_files/standscaler_regression.pkl', 'wb') as file:
    pickle.dump(scaler, file)

# Start Training ANNs Regression Models

In [11]:

# Imports all necessary modules
import tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential    # For sequential models
from tensorflow.keras.layers import Dense         # For nurons or nodes
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard  # TB for display info of some graphs
import datetime

In [13]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1),  # default linear activation function for regression or one nuron
])

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])
model.summary()

e:\End to End ML Project\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
### setup the tensorboard
from tensorflow.keras.callbacks  import TensorBoard, EarlyStopping  #TB Visualize all out logs
log_dir="log/fit_regression" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=1
)

In [15]:
### Setup earlystoping (after some epochs if loss is not decrease the stop it or earlystopping)
early_stopping_callback = EarlyStopping(
    monitor='val_loss', 
    patience=30,
    restore_best_weights=True,
) # At least 5 epochs and monitor val_loss


In [16]:
### Here we train the model

history = model.fit(
    X_train, 
    y_train, 
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 0.2547 - mae: 0.2547 - val_loss: 0.2006 - val_mae: 0.2006
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1975 - mae: 0.1975 - val_loss: 0.1864 - val_mae: 0.1864
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1873 - mae: 0.1873 - val_loss: 0.1801 - val_mae: 0.1801
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1852 - mae: 0.1852 - val_loss: 0.1756 - val_mae: 0.1756
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1762 - mae: 0.1762 - val_loss: 0.1701 - val_mae: 0.1701
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1667 - mae: 0.1667 - val_loss: 0.1682 - val_mae: 0.1682
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1697 - mae: 0.1697 - val_loss: 0.1657 - val_mae: 0.1657
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1703 - mae: 0.1703 - val_loss: 0.1623 - val_mae: 0.1623
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/

In [18]:
%load_ext tensorboard
%tensorboard --logdir=log/fit_regression20250226-230320

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 2308), started 0:00:11 ago. (Use '!kill 2308' to kill it.)